In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:

import os
spectrogram_path = "/content/gdrive/MyDrive/spectrograms"
print(os.listdir(spectrogram_path))

In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/data.csv')
df

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
model = Sequential()
input_shape= (128,128,1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape,padding='same'))
model.add(MaxPooling2D((4, 2), strides=(4, 2),padding="same"))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="same"))
model.add(MaxPooling2D((4, 2), strides=(4, 2),padding="same"))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="same"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

In [ ]:
import numpy as np
X_train_len = int(len(df["X_train"])/(128*128))
Y_train_len = int(len(df["Y_train"])/(128*128))
print("ne",df["Y_train"].to_numpy().shape,Y_train_len)
X_train = np.reshape(df["X_train"].to_numpy(),(X_train_len,128,128,1))
Y_train =df["Y_train"].to_numpy()[:Y_train_len]

from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
callback=[ReduceLROnPlateau(patience=1, verbose=1),
          ModelCheckpoint(filepath="/content/gdrive/MyDrive/Nicegroup_DL_Project.h5", monitor='loss', verbose=1, save_best_only=True)]

#compile
model.compile(optimizer="Adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])

#train
print('training started.... please wait!')
result = model.fit(x=X_train, y=Y_train,
                    epochs=100,
                    batch_size=1, 
                    verbose=0,
                    # validation_data= (X_val, y_val), 
                    callbacks=callback)
# print('training finished')

In [ ]:
from keras.models import load_model
pretrain_model = load_model("/content/gdrive/MyDrive/Nicegroup_DL_Project.h5")
pretrain_model.summary()


In [ ]:
import librosa
import numpy as np
audio_path = "/content/gdrive/MyDrive/UrbanSound8K/audio/fold1/54858-3-1-2.wav"
audiodata, samplerate = librosa.load(audio_path)
spec = librosa.feature.melspectrogram(y=audiodata)
spec_conv = librosa.amplitude_to_db(spec,ref=np.max)


In [ ]:
print(spec_conv.shape)
class_names = [indis for indis in range(0,10)]
import cv2
import tensorflow as tf
cv2.imwrite("/content/gdrive/MyDrive/bak.jpg",spec_conv)

image = tf.keras.preprocessing.image.load_img("/content/gdrive/MyDrive/bak.jpg",color_mode="grayscale",target_size=(128,128))
image = np.expand_dims(image,axis=0)
image = np.array(image)
print("image.shape",image.shape)
cikti = pretrain_model.predict(image)
print("cikti",cikti,len(cikti[0]))
predict_index = np.argmax(cikti)
print(predict_index,class_names[predict_index])
